In [1]:
import requests
import time
import arxiv
import os
from tqdm import tqdm
import pandas as pd

def get_arxiv_pdf(arxiv_id):
    try:
        search = arxiv.Search(id_list=[arxiv_id])
        paper = next(search.results())
        return paper.pdf_url
    except Exception as e:
        print(f"\nError getting arXiv PDF URL for {arxiv_id}: {e}")
        return None

def retrieve_url(url, filepath):
    try:
        response = requests.get(url, stream=True)
        response.raise_for_status()
        with open(filepath, 'wb') as f:
            for chunk in response.iter_content(chunk_size=8192):
                f.write(chunk)
        return True
    except Exception as e:
        print(f"\nError downloading {url}: {e}")
        return False

def get_all_citations(paper_id):
    citations = []
    offset = 0
    limit = 1000
    total_citations = 0
    
    url = f"http://api.semanticscholar.org/graph/v1/paper/{paper_id}"
    params = {"fields": "citationCount"}
    try:
        response = requests.get(url, params=params)
        if response.status_code == 200:
            total_citations = response.json()['citationCount']
            print(f"Total citations to fetch: {total_citations}")
        else:
            print(f"Failed to get citation count: {response.status_code}")
            return citations
    except Exception as e:
        print(f"Error getting citation count: {e}")
        return citations
    
    with tqdm(total=total_citations, desc="Fetching citations") as pbar:
        while True:
            url = f"http://api.semanticscholar.org/graph/v1/paper/{paper_id}/citations"
            params = {
                "fields": "citingPaper.title,citingPaper.externalIds,citingPaper.url",
                "limit": limit,
                "offset": offset
            }
            
            try:
                response = requests.get(url, params=params)
                if response.status_code == 200:
                    data = response.json()
                    batch = data.get('data', [])
                    if not batch:
                        break
                    
                    citations.extend(batch)
                    pbar.update(len(batch))
                    
                    offset += limit
                    time.sleep(1)
                else:
                    print(f"Error fetching citations: {response.status_code}")
                    break
            except Exception as e:
                print(f"Error in citation request: {e}")
                break
                
    return citations

paperId = "1b6e810ce0afd0dd093f789d2b2742d047e316d5"
citations = get_all_citations(paperId)

arxiv_citations = []
for citation in citations:
    if 'citingPaper' in citation:
        paper = citation['citingPaper']
        external_ids = paper.get('externalIds', {})
        if any(key.lower() == 'arxiv' for key in external_ids.keys()):
            arxiv_citations.append(paper)

print(f"\nTotal citations with arXiv IDs: {len(arxiv_citations)}")
print("\nFirst 5 papers to be downloaded:")
for i, paper in enumerate(arxiv_citations[:5]):
    print(f"{i+1}. {paper.get('title')}")

print("\nStarting downloads...")
if not os.path.exists("pdfs"):
    os.makedirs("pdfs")

for paper in tqdm(arxiv_citations, desc="Processing arXiv papers"):
    title = paper['title']
    arxiv_key = next(key for key in paper['externalIds'].keys() if key.lower() == 'arxiv')
    arxiv_id = paper['externalIds'][arxiv_key]
    
    safe_title = "".join(c for c in title if c.isalnum() or c in (' ', '-', '_')).strip()
    pdf_path = f"pdfs/{safe_title}.pdf"
    
    if os.path.exists(pdf_path):
        continue
        
    pdf_url = get_arxiv_pdf(arxiv_id)
    if pdf_url:
        if retrieve_url(pdf_url, pdf_path):
            continue  
        else:
            print(f"\nFailed to download: {title}")
    
    time.sleep(3)

Total citations to fetch: 6318


Fetching citations: 100%|██████████████████| 6318/6318 [00:25<00:00, 252.40it/s]



Total citations with arXiv IDs: 4895

First 5 papers to be downloaded:
1. Enhancing Table Recognition with Vision LLMs: A Benchmark and Neighbor-Guided Toolchain Reasoner
2. Efficiently Serving LLM Reasoning Programs with Certaindex
3. Do NOT Think That Much for 2+3=? On the Overthinking of o1-Like LLMs
4. Pushing the Envelope of Low-Bit LLM via Dynamic Error Compensation
5. Toward Adaptive Reasoning in Large Language Models with Thought Rollback

Starting downloads...


Processing arXiv papers:   0%|                         | 0/4895 [00:00<?, ?it/s]/var/folders/0q/6k0gk6g5281gcsvh418ty0140000gn/T/ipykernel_2060/2173061590.py:10: DeprecationWarning: The 'Search.results' method is deprecated, use 'Client.results' instead
  paper = next(search.results())
Processing arXiv papers:  10%|█▌             | 502/4895 [05:29<41:40,  1.76it/s]


Error downloading http://arxiv.org/pdf/2410.07062v2: 404 Client Error: Not Found for url: http://arxiv.org/pdf/2410.07062v2

Failed to download: TinyEmo: Scaling down Emotional Reasoning via Metric Projection


...

Processing arXiv papers:  80%|███████████▏  | 3922/4895 [54:26<14:02,  1.15it/s]


Error downloading http://arxiv.org/pdf/2306.08997v2: 404 Client Error: NOT FOUND for url: http://arxiv.org/pdf/2306.08997v2

Failed to download: Exploring the MIT Mathematics and EECS Curriculum Using Large Language Models


Processing arXiv papers:  95%|███████████▍| 4667/4895 [1:10:46<05:45,  1.52s/it]


Error downloading http://arxiv.org/pdf/2306.07622v2: 404 Client Error: Not Found for url: http://arxiv.org/pdf/2306.07622v2

Failed to download: Human-like intuitive behavior and reasoning biases emerged in large language models but disappeared in ChatGPT


Processing arXiv papers: 100%|████████████| 4895/4895 [1:14:17<00:00,  1.10it/s]


In [2]:
from pathlib import Path
from PyPDF2 import PdfReader
import json
from tqdm import tqdm
import warnings
import logging

logger = logging.getLogger("PyPDF2")
logger.setLevel(logging.ERROR)

warnings.filterwarnings('ignore', message='.*FloatObject.*invalid.*')
warnings.filterwarnings('ignore', message='.*unknown widths.*')
warnings.filterwarnings('ignore', category=Warning, module='PyPDF2')

def clean_text(text):
    """Clean text of invalid Unicode characters"""
    if not text:
        return ""
    # Replace surrogate characters with a replacement character
    return text.encode('utf-16', 'surrogatepass').decode('utf-16', 'replace')

directory = "pdfs"
pdf_files = list(Path(directory).glob('*.pdf'))
pdf_text_dict = {}

# Create progress bar
for file in tqdm(pdf_files, desc="Processing PDFs"):
    try:
        reader = PdfReader(file)
        meta = reader.metadata
        
        # Initialize dictionary entry with metadata, cleaning each field
        pdf_text_dict[file.stem] = {
            "filename": file.name,
            "author": clean_text(meta.author) if meta.author else "",
            "creator": clean_text(meta.creator) if meta.creator else "",
            "subject": clean_text(meta.subject) if meta.subject else "",
            "title": clean_text(meta.title) if meta.title else "",
            "text": ""
        }
        
        # Extract text from all pages
        text_content = []
        for page in reader.pages:
            try:
                extracted_text = page.extract_text()
                if extracted_text:
                    text_content.append(clean_text(extracted_text))
            except Exception as page_error:
                print(f"\nError extracting text from page in {file}: {str(page_error)}")
                continue
            
        # Join all pages' text with newlines
        pdf_text_dict[file.stem]["text"] = "\n".join(text_content)
        
    except Exception as e:
        print(f"\nError processing {file}: {str(e)}")
        pdf_text_dict[file.stem] = {
            "filename": file.name,
            "error": str(e),
            "text": ""
        }

# Save to JSON file
output_path = "pdf_contents.json"
try:
    # First try with standard encoding
    try:
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(pdf_text_dict, f, ensure_ascii=False, indent=4)
    except UnicodeEncodeError:
        # If that fails, fall back to ensuring ASCII with escaping
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(pdf_text_dict, f, ensure_ascii=True, indent=4)
    print(f"\nSuccessfully saved data to {output_path}")
except Exception as e:
    print(f"\nError saving JSON file: {str(e)}")
    # Last resort: try to save with problematic characters removed
    try:
        cleaned_dict = {k: {
            key: str(value).encode('ascii', 'ignore').decode('ascii') 
            for key, value in v.items()
        } for k, v in pdf_text_dict.items()}
        with open(output_path, 'w', encoding='utf-8') as f:
            json.dump(cleaned_dict, f, ensure_ascii=True, indent=4)
        print(f"Saved file with ASCII-only characters to {output_path}")
    except Exception as final_e:
        print(f"Final attempt to save failed: {str(final_e)}")

Processing PDFs: 100%|████████████████████| 4886/4886 [1:00:14<00:00,  1.35it/s]



Successfully saved data to pdf_contents.json


In [5]:
import json
import openai
from pathlib import Path
from tqdm import tqdm
import time
from openai import OpenAI
import tiktoken
from datetime import datetime

# Add your OpenAI API token here

def count_tokens(text):
    encoding = tiktoken.encoding_for_model("gpt-4o")
    return len(encoding.encode(text))

def truncate_text(text, max_tokens=14000):
    encoding = tiktoken.encoding_for_model("gpt-4o")
    tokens = encoding.encode(text)
    if len(tokens) > max_tokens:
        tokens = tokens[:max_tokens]
        text = encoding.decode(tokens)
    return text

def create_prompt(paper_text):
    return f"""Analyze this academic paper and extract all benchmarks/datasets and models mentioned in experiments, evaluations, or comparisons.

Rules for extraction:
1. For benchmarks/datasets:
   - Include standard evaluation datasets (e.g., MNIST, ImageNet, SQuAD)
   - Include custom datasets if they're used for evaluation
   - Do NOT include training datasets unless they're also used for evaluation

2. For models:
   - Include baseline models used for comparison
   - Include proposed/novel models being evaluated
   - Include model variants tested in ablation studies
   - Do NOT include referenced models that weren't actually tested

Format your response as a JSON object with this exact structure:
{{
    "benchmarks": ["benchmark1", "benchmark2"],
    "models": ["model1", "model2"]
}}

Paper text:
{paper_text}"""

def analyze_paper_with_gpt(paper_text):
    max_retries = 3
    for attempt in range(max_retries):
        try:
            truncated_text = truncate_text(paper_text)
            was_truncated = len(truncated_text) < len(paper_text)
            
            prompt = create_prompt(truncated_text)
            token_count = count_tokens(prompt)
            
            if token_count > 15000:
                print("Warning: Token count exceeds safe limit")
                return {
                    "benchmarks": [],
                    "models": [],
                    "error": "Token limit exceeded"
                }
                
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": "You are a helpful assistant that analyzes academic papers and returns responses in JSON format only."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0,
                max_tokens=1000,
                response_format={"type": "json_object"}
            )

            result = response.choices[0].message.content
            
            # Add error checking for response content
            if not result.strip().startswith('{') or not result.strip().endswith('}'):
                raise json.JSONDecodeError("Invalid JSON format", result, 0)
                
            parsed_result = json.loads(result)
            
            if was_truncated:
                parsed_result["note"] = "Analysis based on truncated paper text"
                
            return parsed_result

        except json.JSONDecodeError as e:
            print(f"\nAttempt {attempt + 1}/{max_retries} - JSON parsing error: {str(e)}")
            print(f"Raw response: {result}")
            if attempt == max_retries - 1:
                return {
                    "benchmarks": [],
                    "models": [],
                    "error": f"Failed to parse GPT response as JSON after {max_retries} attempts: {str(e)}"
                }
            time.sleep(2)  # Wait before retry
            
        except Exception as e:
            print(f"\nAttempt {attempt + 1}/{max_retries} - Error: {str(e)}")
            if attempt == max_retries - 1:
                return {
                    "benchmarks": [],
                    "models": [],
                    "error": str(e)
                }
            time.sleep(2)  # Wait before retry

def save_progress(analysis_results, base_filename, iteration):
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{base_filename}_iter{iteration}_{timestamp}.json"
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(analysis_results, f, ensure_ascii=False, indent=4)
    print(f"\nProgress saved to {filename}")

def main():
    input_file = "pdf_contents.json"
    output_base = "paper_analysis"

    with open(input_file, 'r', encoding='utf-8') as f:
        pdf_contents = json.load(f)

    analysis_results = {}
    
    # Convert items to list for tqdm
    items = list(pdf_contents.items())
    save_interval = 500  # Save every 500 papers
    
    for i, (paper_id, paper_data) in enumerate(tqdm(items[3000:], desc="Analyzing papers")):
        paper_text = paper_data.get('text', '')
        
        if not paper_text:
            print(f"\nSkipping {paper_id} - no text content")
            continue

        try:
            result = analyze_paper_with_gpt(paper_text)
            analysis_results[paper_id] = {
                "filename": paper_data['filename'],
                "analysis": result
            }
            
            # Save progress every save_interval papers
            if (i + 1) % save_interval == 0:
                save_progress(analysis_results, output_base, f"checkpoint_{i+1}")
                
            time.sleep(2)

        except Exception as e:
            print(f"\nError analyzing {paper_id}: {str(e)}")
            analysis_results[paper_id] = {
                "filename": paper_data['filename'],
                "error": str(e)
            }

    # Save final results
    save_progress(analysis_results, output_base, "final")

if __name__ == "__main__":
    main()

Analyzing papers:   3%|▎         | 48/1886 [04:20<2:44:35,  5.37s/it]


Attempt 1/3 - Error: Encountered text corresponding to disallowed special token '<|endoftext|>'.
If you want this text to be encoded as a special token, pass it to `allowed_special`, e.g. `allowed_special={'<|endoftext|>', ...}`.
If you want this text to be encoded as normal text, disable the check for this token by passing `disallowed_special=(enc.special_tokens_set - {'<|endoftext|>'})`.
To disable this check for all special tokens, pass `disallowed_special=()`.


Attempt 2/3 - Error: Encountered text corresponding to disallowed special token '<|endoftext|>'.
If you want this text to be encoded as a special token, pass it to `allowed_special`, e.g. `allowed_special={'<|endoftext|>', ...}`.
If you want this text to be encoded as normal text, disable the check for this token by passing `disallowed_special=(enc.special_tokens_set - {'<|endoftext|>'})`.
To disable this check for all special tokens, pass `disallowed_special=()`.


Attempt 3/3 - Error: Encountered text corresponding to di

Analyzing papers:   4%|▍         | 72/1886 [07:24<3:53:16,  7.72s/it]


Attempt 1/3 - JSON parsing error: Invalid JSON format: line 1 column 1 (char 0)
Raw response: {
    "benchmarks": [
        "AQA-Bench",
        "GuessNum",
        "DFS",
        "BFS",
        "Coin",
        "CaveDFS",
        "CaveBFS"
    ],
    "models": [
        "GPT-4",
        "Gemini",
        "GPT-3.5-Turbo",
        "GPT-4-Turbo",
        "Gemini-Pro",
        "


Analyzing papers:   9%|▊         | 161/1886 [20:59<2:14:33,  4.68s/it]


Attempt 1/3 - JSON parsing error: Invalid JSON format: line 1 column 1 (char 0)
Raw response: {
    "benchmarks": [
        "ACE04",
        "ACE05",
        "CoNLL03",
        "OntoNotes 5.0",
        "GENIA",
        "NYT",
        "ADE",
        "CoNLL04",
        "SciERC",
        "TACRED",
        "Re-TACRED",
        "TACREV",
        "SemEval"
    ],
    "models": [
        "GPT-4",
        "ChatGPT",
        "LLaMA",
        "Flan-T5",
        "CodeX",
        "GPT-NER",
        "Cp-NER",
        "LLMaAA",
        "PromptNER",
        "UniNER",
        "NAG-NER",
        "GNER",
        "NuNER",
        "MetaNER",
        "LinkNER",
        "SLCoLM",
        "ProgGen",
        "C-ICL",
        "VerifiNER",
        "ConsistNER",
        "GLiNER",
        "LTNER",
        "ToNER",
        "RT",
        "VANER",
        "RiVEG",
        "LLM-DA",
        "REBEL",
        "QA4RE",
        "GPT-RE",
        "STAR",
        "AugURE",
        "REPAL",
        "RAG4RE",
        "BART-

Analyzing papers:   9%|▉         | 176/1886 [34:42<5:37:17, 11.83s/it]


KeyboardInterrupt: 

In [11]:
import json
import glob

json_files = glob.glob('paper_analysis_itercheckpoint_*.json')
combined_data = {}

for file in json_files:
    with open(file) as f:
        data = json.load(f)
        combined_data.update(data)

with open('combined_analysis.json', 'w') as f:
    json.dump(combined_data, f, indent=2)


In [1]:
import pandas as pd

excel_file = "ensemble_works-new.xlsx"
sheet_names = pd.read_excel(excel_file, sheet_name=None).keys()
print(sheet_names)

dict_keys(['Properties', 'Models Used', 'Models Used Recently', 'Models Used Paste', 'Benchmarks Used', 'Benchmarks by Use'])


In [9]:
model_df = pd.read_excel(excel_file, sheet_name='Models Used')
model_df = model_df.iloc[:109, :-2]
# model_df.head()
# model_df.tail()

In [10]:
benchmark_df = pd.read_excel(excel_file, sheet_name='Benchmarks Used')
benchmark_df = benchmarks_df.iloc[:109, :-5]

In [12]:
annoted_papers = {}

for (index_one, model_row), (index_two, benchmark_row) in zip(model_df.iterrows(), benchmark_df.iterrows()):
    paper_title = model_row['Paper Titles']
    annoted_papers[paper_title] = {
        "filename": f"{paper_title}.pdf",
        "analysis": {
            "models": [
                model for model, value in model_row[2:].items()
                if value == 1.0
            ],
            "benchmarks": [
                benchmark for benchmark, value in benchmark_row[2:].items()
                if value == 1.0
            ],
            "note": "Analysis based on human review."
        }
    }

with open('annotated_papers.json', 'w', encoding='utf-8') as file:
    json.dump(annoted_papers, file, indent=2)

In [25]:
import json

def clean_title(title):
    # Remove common punctuation and convert to lowercase
    return title.lower().replace(":", "").replace("-", "").replace("  ", " ").strip()

with open('annotated_papers.json', 'r') as file1, open('pdf_contents.json', 'r') as file2:
    data1 = json.load(file1)
    data2 = json.load(file2)
    
    matches = 0
    
    # For each paper in first file
    for title1, paper1 in data1.items():
        title1_clean = clean_title(title1)
        
        # Check if this title exists in any title from second file
        for title2 in data2.keys():
            title2_clean = clean_title(title2)
            
            # Check if titles are effectively the same
            if title1_clean == title2_clean or \
               title1_clean in title2_clean and len(title1_clean) > 20:  # Length check to avoid short title false matches
                matches += 1
                print(f"\nMatch found:")
                print(f"File 1: {title1}")
                print(f"File 2: {title2}")
                break

    print(f"\nTotal papers in first file: {len(data1)}")
    print(f"Total papers in second file: {len(data2)}")
    print(f"Number of matching papers: {matches}")


Match found:
File 1: ReAct: Synergizing Reasoning and Acting in Language Models
File 2: ReAct Synergizing Reasoning and Acting in Language Models

Match found:
File 1: Tree of Thoughts: Deliberate Problem Solving with Large Language Models
File 2: Tree of Thoughts Deliberate Problem Solving with Large Language Models

Match found:
File 1: Boosted Prompt Ensembles for Large Language Models
File 2: Boosted Prompt Ensembles for Large Language Models

Match found:
File 1: Ask Me Anything: A simple strategy for prompting language models
File 2: Ask Me Anything A simple strategy for prompting language models

Match found:
File 1: Graph of Thoughts: Solving Elaborate Problems with Large Language Models
File 2: Graph of Thoughts Solving Elaborate Problems with Large Language Models

Match found:
File 1: More Agents Is All You Need
File 2: More Agents Is All You Need

Match found:
File 1: ReConcile: Round-Table Conference Improves Reasoning via Consensus among Diverse LLMs
File 2: ReConcile Ro

In [39]:
import json
import openai
from pathlib import Path
from tqdm import tqdm
import time
from openai import OpenAI
import tiktoken
from datetime import datetime

# Replace with your API key

def count_tokens(text):
    encoding = tiktoken.encoding_for_model("gpt-4o")
    return len(encoding.encode(text))

def truncate_text(text, max_tokens=14000):
    encoding = tiktoken.encoding_for_model("gpt-4o")
    tokens = encoding.encode(text)
    if len(tokens) > max_tokens:
        tokens = tokens[:max_tokens]
        text = encoding.decode(tokens)
    return text

def create_prompt(paper_text):
    return f"""Analyze this academic paper and extract two specific types of information:

1. Benchmarks/Datasets used for evaluation:
   - Include standard evaluation datasets (e.g., MNIST, ImageNet, SQuAD)
   - Include custom datasets if they're used for evaluation
   - Do NOT include training datasets unless they're also used for evaluation

2. Base Language Models used in experiments:
   - Include specific model architectures and variants (e.g., GPT-4, LLaMA-70B, PaLM-540B), which includes parameter size
   - Do NOT include methods or techniques (e.g., don't include Chain-of-Thought, Self-Consistency, etc.)
   - For custom models, specify the base model they use (e.g., if a paper introduces "CustomBERT", note it uses BERT as base)

Format your response as a JSON object with this exact structure:
{{
    "benchmarks": ["benchmark1", "benchmark2"],
    "base_models": ["model1 (with size if specified)", "model2 (with size if specified)"]
}}

Paper text:
{paper_text}"""

def analyze_paper_with_gpt(paper_text):
    max_retries = 3
    for attempt in range(max_retries):
        try:
            truncated_text = truncate_text(paper_text)
            was_truncated = len(truncated_text) < len(paper_text)
            
            prompt = create_prompt(truncated_text)
            token_count = count_tokens(prompt)
            
            if token_count > 15000:
                print("Warning: Token count exceeds safe limit")
                return {
                    "benchmarks": [],
                    "models": [],
                    "error": "Token limit exceeded"
                }
                
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": "You are a helpful graduate research assistant that analyzes academic papers and returns responses in JSON format only."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0,
                max_tokens=1000,
                response_format={"type": "json_object"}
            )

            result = response.choices[0].message.content
            
            # Add error checking for response content
            if not result.strip().startswith('{') or not result.strip().endswith('}'):
                raise json.JSONDecodeError("Invalid JSON format", result, 0)
                
            parsed_result = json.loads(result)
            
            if was_truncated:
                parsed_result["note"] = "Analysis based on truncated paper text"
                
            return parsed_result

        except json.JSONDecodeError as e:
            print(f"\nAttempt {attempt + 1}/{max_retries} - JSON parsing error: {str(e)}")
            print(f"Raw response: {result}")
            if attempt == max_retries - 1:
                return {
                    "benchmarks": [],
                    "models": [],
                    "error": f"Failed to parse GPT response as JSON after {max_retries} attempts: {str(e)}"
                }
            time.sleep(2)  # Wait before retry
            
        except Exception as e:
            print(f"\nAttempt {attempt + 1}/{max_retries} - Error: {str(e)}")
            if attempt == max_retries - 1:
                return {
                    "benchmarks": [],
                    "models": [],
                    "error": str(e)
                }
            time.sleep(2)  # Wait before retry

def save_progress(analysis_results, base_filename, iteration):
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{base_filename}_iter{iteration}_{timestamp}.json"
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(analysis_results, f, ensure_ascii=False, indent=4)
    print(f"\nProgress saved to {filename}")

def clean_title(title):
    return title.lower().replace(":", "").replace("-", "").replace("  ", " ").strip()

def main():
    # First, load and find matching papers
    with open('annotated_papers.json', 'r') as file1, open('pdf_contents.json', 'r') as file2:
        data1 = json.load(file1)
        pdf_contents = json.load(file2)
        
        matched_papers = {}
        
        # Find matching papers
        for title1, paper1 in data1.items():
            title1_clean = clean_title(title1)
            
            for paper_id, paper_data in pdf_contents.items():
                title2 = paper_data['filename']
                title2_clean = clean_title(title2)
                
                if title1_clean == title2_clean or \
                   (title1_clean in title2_clean and len(title1_clean) > 20):
                    matched_papers[paper_id] = paper_data
                    print(f"Match found: {title1}")
                    break

    print(f"\nFound {len(matched_papers)} matching papers to analyze")
    
    # Now process only the matched papers
    analysis_results = {}
    save_interval = 10  # Reduced save interval since we have fewer papers
    
    # Convert items to list for tqdm
    items = list(matched_papers.items())
    
    for i, (paper_id, paper_data) in enumerate(tqdm(items, desc="Analyzing matched papers")):
        paper_text = paper_data.get('text', '')
        
        if not paper_text:
            print(f"\nSkipping {paper_id} - no text content")
            continue

        try:
            result = analyze_paper_with_gpt(paper_text)
            analysis_results[paper_id] = {
                "filename": paper_data['filename'],
                "analysis": result
            }
            
            # Save progress more frequently since we're processing fewer papers
            if (i + 1) % save_interval == 0:
                save_progress(analysis_results, "matched_papers_analysis", f"checkpoint_{i+1}")
                
            time.sleep(2)  # Rate limiting

        except Exception as e:
            print(f"\nError analyzing {paper_id}: {str(e)}")
            analysis_results[paper_id] = {
                "filename": paper_data['filename'],
                "error": str(e)
            }

    # Save final results
    save_progress(analysis_results, "matched_papers_analysis", "final")

if __name__ == "__main__":
    main()

Match found: ReAct: Synergizing Reasoning and Acting in Language Models
Match found: Tree of Thoughts: Deliberate Problem Solving with Large Language Models
Match found: Boosted Prompt Ensembles for Large Language Models
Match found: Ask Me Anything: A simple strategy for prompting language models
Match found: Graph of Thoughts: Solving Elaborate Problems with Large Language Models
Match found: More Agents Is All You Need
Match found: ReConcile: Round-Table Conference Improves Reasoning via Consensus among Diverse LLMs
Match found: Boosting of Thoughts: Trial-and-Error Problem Solving with Large Language Models
Match found: Fill in the Blank: Exploring and Enhancing LLM Capabilities for Backward Reasoning in Math Word Problems
Match found: InferFix: End-to-End Program Repair with LLMs
Match found: Revisit Input Perturbation Problems for LLMs: A Unified Robustness Evaluation Framework for Noisy Slot Filling Task
Match found: FreshLLMs: Refreshing Large Language Models with Search Engine

Analyzing matched papers:  18%|███▏              | 9/50 [01:05<07:29, 10.97s/it]


Progress saved to matched_papers_analysis_itercheckpoint_10_20250103_132347.json


Analyzing matched papers:  38%|██████▍          | 19/50 [01:47<02:18,  4.46s/it]


Progress saved to matched_papers_analysis_itercheckpoint_20_20250103_132430.json


...


Progress saved to matched_papers_analysis_itercheckpoint_40_20250103_132559.json


Analyzing matched papers:  98%|████████████████▋| 49/50 [04:13<00:05,  5.51s/it]


Progress saved to matched_papers_analysis_itercheckpoint_50_20250103_132658.json


Analyzing matched papers: 100%|█████████████████| 50/50 [04:20<00:00,  5.21s/it]


Progress saved to matched_papers_analysis_iterfinal_20250103_132700.json


In [4]:
import json
import openai
from pathlib import Path
from tqdm import tqdm
import time
from openai import OpenAI
import tiktoken
from datetime import datetime
import logging

# Disable OpenAI logging
logging.getLogger("openai").setLevel(logging.WARNING)
logging.getLogger("httpx").setLevel(logging.WARNING)

# Replace with your API key

def count_tokens(text):
    encoding = tiktoken.encoding_for_model("gpt-4o")
    return len(encoding.encode(text))

def truncate_text(text, max_tokens=14000):
    encoding = tiktoken.encoding_for_model("gpt-4o")
    tokens = encoding.encode(text)
    if len(tokens) > max_tokens:
        tokens = tokens[:max_tokens]
        text = encoding.decode(tokens)
    return text

def create_prompt(paper_text):
    return f"""Analyze this academic paper and extract two specific types of information:

1. Benchmarks/Datasets used for evaluation:
   - Include standard evaluation datasets (e.g., MNIST, ImageNet, SQuAD)
   - Include custom datasets if they're used for evaluation
   - Do NOT include training datasets unless they're also used for evaluation

2. Base Language Models used in experiments:
   - Include specific model architectures and variants (e.g., GPT-4, LLaMA-70B, PaLM-540B), which includes parameter size
   - Do NOT include methods or techniques (e.g., don't include Chain-of-Thought, Self-Consistency, etc.)
   - For custom models, specify the base model they use (e.g., if a paper introduces "CustomBERT", note it uses BERT as base)

Format your response as a JSON object with this exact structure:
{{
    "benchmarks": ["benchmark1", "benchmark2"],
    "base_models": ["model1 (with size if specified)", "model2 (with size if specified)"]
}}

Paper text:
{paper_text}"""

def analyze_paper_with_gpt(paper_text):
    max_retries = 3
    for attempt in range(max_retries):
        try:
            truncated_text = truncate_text(paper_text)
            was_truncated = len(truncated_text) < len(paper_text)
            
            prompt = create_prompt(truncated_text)
            token_count = count_tokens(prompt)
            
            if token_count > 15000:
                print("Warning: Token count exceeds safe limit")
                return {
                    "benchmarks": [],
                    "models": [],
                    "error": "Token limit exceeded"
                }
                
            response = client.chat.completions.create(
                model="gpt-4o",
                messages=[
                    {"role": "system", "content": "You are a helpful graduate research assistant that analyzes academic papers and returns responses in JSON format only."},
                    {"role": "user", "content": prompt}
                ],
                temperature=0,
                max_tokens=1000,
                response_format={"type": "json_object"}
            )

            result = response.choices[0].message.content
            
            # Add error checking for response content
            if not result.strip().startswith('{') or not result.strip().endswith('}'):
                raise json.JSONDecodeError("Invalid JSON format", result, 0)
                
            parsed_result = json.loads(result)
            
            if was_truncated:
                parsed_result["note"] = "Analysis based on truncated paper text"
                
            return parsed_result

        except json.JSONDecodeError as e:
            print(f"\nAttempt {attempt + 1}/{max_retries} - JSON parsing error: {str(e)}")
            print(f"Raw response: {result}")
            if attempt == max_retries - 1:
                return {
                    "benchmarks": [],
                    "models": [],
                    "error": f"Failed to parse GPT response as JSON after {max_retries} attempts: {str(e)}"
                }
            time.sleep(2)  # Wait before retry
            
        except Exception as e:
            print(f"\nAttempt {attempt + 1}/{max_retries} - Error: {str(e)}")
            if attempt == max_retries - 1:
                return {
                    "benchmarks": [],
                    "models": [],
                    "error": str(e)
                }
            time.sleep(2)  # Wait before retry

def save_progress(analysis_results, base_filename, iteration):
    timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
    filename = f"{base_filename}_iter{iteration}_{timestamp}.json"
    with open(filename, 'w', encoding='utf-8') as f:
        json.dump(analysis_results, f, ensure_ascii=False, indent=4)
    print(f"\nProgress saved to {filename}")

def main():
    # Load all papers from pdf_contents
    with open('pdf_contents.json', 'r') as file:
        pdf_contents = json.load(file)

    print(f"\nFound {len(pdf_contents)} papers to analyze")
    
    # Initialize results dictionary
    analysis_results = {}
    save_interval = 20  # Save every 20 papers
    
    # Convert items to list for tqdm
    items = list(pdf_contents.items())
    
    for i, (paper_id, paper_data) in enumerate(tqdm(items, desc="Analyzing papers")):
        paper_text = paper_data.get('text', '')
        
        if not paper_text:
            print(f"\nSkipping {paper_id} - no text content")
            continue

        try:
            result = analyze_paper_with_gpt(paper_text)
            analysis_results[paper_id] = {
                "filename": paper_data['filename'],
                "analysis": result
            }
            
            # Save progress periodically
            if (i + 1) % save_interval == 0:
                save_progress(analysis_results, "papers_analysis", f"checkpoint_{i+1}")
                
            time.sleep(2)  # Rate limiting

        except Exception as e:
            print(f"\nError analyzing {paper_id}: {str(e)}")
            analysis_results[paper_id] = {
                "filename": paper_data['filename'],
                "error": str(e)
            }

    # Save final results
    save_progress(analysis_results, "papers_analysis", "final")

if __name__ == "__main__":
    main()


Found 4886 papers to analyze


Analyzing papers:   0%|                     | 19/4886 [01:22<6:03:38,  4.48s/it]


Progress saved to papers_analysis_itercheckpoint_20_20250107_140550.json


Analyzing papers:   1%|                     | 25/4886 [01:55<6:22:50,  4.73s/it]


Attempt 1/3 - Error: Encountered text corresponding to disallowed special token '<|endoftext|>'.
If you want this text to be encoded as a special token, pass it to `allowed_special`, e.g. `allowed_special={'<|endoftext|>', ...}`.
If you want this text to be encoded as normal text, disable the check for this token by passing `disallowed_special=(enc.special_tokens_set - {'<|endoftext|>'})`.
To disable this check for all special tokens, pass `disallowed_special=()`.


Attempt 2/3 - Error: Encountered text corresponding to disallowed special token '<|endoftext|>'.
If you want this text to be encoded as a special token, pass it to `allowed_special`, e.g. `allowed_special={'<|endoftext|>', ...}`.
If you want this text to be encoded as normal text, disable the check for this token by passing `disallowed_special=(enc.special_tokens_set - {'<|endoftext|>'})`.
To disable this check for all special tokens, pass `disallowed_special=()`.


Attempt 3/3 - Error: Encountered text corresponding to di

...


Progress saved to papers_analysis_itercheckpoint_4860_20250107_200604.json


Analyzing papers: 100%|██████████████████▉| 4879/4886 [6:03:11<00:30,  4.31s/it]


Progress saved to papers_analysis_itercheckpoint_4880_20250107_200733.json


Analyzing papers: 100%|███████████████████| 4886/4886 [6:03:41<00:00,  4.47s/it]



Progress saved to papers_analysis_iterfinal_20250107_200800.json


In [4]:
import json
from collections import Counter
from typing import Dict, Any
import pandas as pd
import re

def normalize_model_name(name: str) -> str:
    """
    Normalizes model names to handle different variations.
    
    Args:
        name: Original model name
        
    Returns:
        Normalized model name
    """
    # Remove quotation marks and extra spaces
    name = name.strip('"').strip()
    
    # Convert to lowercase for initial processing
    normalized = name.lower()
    
    # Remove extra spaces between parts
    normalized = re.sub(r'\s+', ' ', normalized)
    
    # Standardize common variations while preserving distinct models
    replacements = {
        'gpt-3.5-turbo': ['gpt-3.5 turbo', 'gpt3.5-turbo', 'gpt3.5 turbo', 'gpt-3.5-t', 'chatgpt', 'chat-gpt', 'chat gpt'],  # ChatGPT uses GPT-3.5-turbo
        'gpt-3.5': ['gpt3.5', 'gpt 3.5'],  # Keep base GPT-3.5 separate
        'gpt-4': ['gpt4', 'gpt 4', 'gpt-4-turbo'],  # Group GPT-4 variations
        'llama-2': ['llama2', 'llama-2', 'llama 2'],  # Standardize LLaMA 2 naming
        'llama': ['llama1', 'llama 1'],  # Keep original LLaMA separate
        'roberta': ['roberta-base', 'roberta base'],
        'bert': ['bert-base', 'bert base'],
    }
    
    # Apply replacements
    for standard, variants in replacements.items():
        if normalized in variants or normalized == standard:
            return standard
            
    # Standardize separators
    normalized = re.sub(r'[-_\s]+', '-', normalized)
    
    return normalized

def analyze_paper_data(data: Dict[str, Any]) -> tuple[Counter, Counter]:
    """
    Recursively analyzes paper data to count unique models and benchmarks.
    """
    model_counter = Counter()
    benchmark_counter = Counter()
    
    def process_item(item):
        if isinstance(item, dict):
            # Handle base_models section
            if 'base_models' in item and isinstance(item['base_models'], list):
                for model in item['base_models']:
                    if isinstance(model, str):
                        normalized_name = normalize_model_name(model)
                        model_counter[normalized_name] += 1
            
            # Handle benchmarks section
            if 'benchmarks' in item and isinstance(item['benchmarks'], list):
                for benchmark in item['benchmarks']:
                    if isinstance(benchmark, str):
                        normalized_name = benchmark.strip('"').strip().lower()
                        normalized_name = re.sub(r'[-_\s]+', '-', normalized_name)
                        benchmark_counter[normalized_name] += 1
            
            # Recursively process all dictionary values
            for value in item.values():
                process_item(value)
                
        elif isinstance(item, list):
            # Recursively process all list items
            for value in item:
                process_item(value)
    
    process_item(data)
    return model_counter, benchmark_counter

def create_frequency_dataframes(model_counter: Counter, benchmark_counter: Counter) -> tuple[pd.DataFrame, pd.DataFrame]:
    """
    Creates pandas DataFrames with frequency information.
    """
    # Create DataFrame for models
    models_df = pd.DataFrame([
        {"model": model, "frequency": count}
        for model, count in model_counter.most_common()
    ])
    
    # Create DataFrame for benchmarks
    benchmarks_df = pd.DataFrame([
        {"benchmark": benchmark, "frequency": count}
        for benchmark, count in benchmark_counter.most_common()
    ])
    
    return models_df, benchmarks_df

def print_statistics(models_df: pd.DataFrame, benchmarks_df: pd.DataFrame):
    """
    Prints detailed statistics about models and benchmarks.
    """
    print("=== Model Statistics ===")
    print(f"Total unique models (after normalization): {len(models_df)}")
    print(f"Total model mentions: {models_df['frequency'].sum()}")
    print("\nTop 20 most frequently mentioned models:")
    print(models_df.head(20).to_string(index=False))
    
    print("\n=== Benchmark Statistics ===")
    print(f"Total unique benchmarks: {len(benchmarks_df)}")
    print(f"Total benchmark mentions: {benchmarks_df['frequency'].sum()}")
    print("\nTop 20 most frequently mentioned benchmarks:")
    print(benchmarks_df.head(20).to_string(index=False))

def main():
    try:
        # Load and process data
        with open('papers_analysis_iterfinal_20250107_200800.json', 'r') as f:
            data = json.load(f)
        
        # Get frequency counts
        model_counter, benchmark_counter = analyze_paper_data(data)
        
        # Create DataFrames
        models_df, benchmarks_df = create_frequency_dataframes(model_counter, benchmark_counter)
        
        # Save frequency data to CSV
        models_df.to_csv('model_frequencies_normalized.csv', index=False)
        benchmarks_df.to_csv('benchmark_frequencies_normalized.csv', index=False)
        
        # Print statistics
        print_statistics(models_df, benchmarks_df)
        
        print("\nDetailed frequency data has been saved to:")
        print("- model_frequencies_normalized.csv")
        print("- benchmark_frequencies_normalized.csv")
        
    except FileNotFoundError:
        print("Error: papers_analysis_iterfinal_20250107_200800.json file not found")
    except json.JSONDecodeError:
        print("Error: Invalid JSON format in the input file")
    except Exception as e:
        print(f"An unexpected error occurred: {str(e)}")

if __name__ == "__main__":
    main()

=== Model Statistics ===
Total unique models (after normalization): 4809
Total model mentions: 16665

Top 20 most frequently mentioned models:
        model  frequency
        gpt-4       1579
gpt-3.5-turbo        743
      gpt-3.5        530
       gpt-4o        251
        gpt-3        250
         bert        198
   mistral-7b        183
   llama-2-7b        165
     llama-7b        140
        llama        140
    llama2-7b        134
       gpt-4v        120
      llama-2        109
      roberta        108
  llama-2-13b        102
 gpt-3-(175b)         98
  llama-2-70b         95
        gpt-2         90
   vicuna-13b         86
           t5         78

=== Benchmark Statistics ===
Total unique benchmarks: 7635
Total benchmark mentions: 15013

Top 20 most frequently mentioned benchmarks:
    benchmark  frequency
        gsm8k        424
         math        175
         mmlu        171
        svamp        139
   strategyqa        126
     hotpotqa        120
    humaneval      

In [1]:
import json
import arxiv
import time
import re
from datetime import datetime
from tqdm import tqdm

def clean_title(title):
    """Clean and normalize the title for better matching"""
    # Remove special characters and extra whitespace
    cleaned = re.sub(r'[^\w\s-]', '', title)
    cleaned = ' '.join(cleaned.split())
    return cleaned

def fetch_arxiv_date(title):
    """
    Fetch publication date from arXiv given a paper title.
    Try multiple search strategies.
    """
    client = arxiv.Client()
    
    # Strategy 1: Exact title match
    search = arxiv.Search(
        query=f'ti:"{title}"',
        max_results=1
    )
    result = next(client.results(search), None)
    if result:
        return result.published.strftime('%Y-%m-%d'), result.entry_id

    time.sleep(1)  # Rate limiting between attempts
    
    # Strategy 2: Search without quotes and ti: prefix
    cleaned_title = clean_title(title)
    search = arxiv.Search(
        query=cleaned_title,
        max_results=5  # Get more results to find potential matches
    )
    
    # Check each result for title similarity
    results = list(client.results(search))
    for result in results:
        result_title = clean_title(result.title)
        # Check if titles are very similar
        if (cleaned_title.lower() in result_title.lower() or 
            result_title.lower() in cleaned_title.lower()):
            return result.published.strftime('%Y-%m-%d'), result.entry_id
    
    time.sleep(1)  # Rate limiting between attempts
    
    # Strategy 3: Try with first few words of title
    first_words = ' '.join(cleaned_title.split()[:4])  # First 4 words
    search = arxiv.Search(
        query=f'ti:"{first_words}"',
        max_results=5
    )
    
    results = list(client.results(search))
    for result in results:
        result_title = clean_title(result.title)
        if cleaned_title.lower() in result_title.lower():
            return result.published.strftime('%Y-%m-%d'), result.entry_id
            
    return None, None

# Load your JSON data
with open('papers_analysis_iterfinal_20250107_200800.json', 'r') as f:
    papers_data = json.load(f)

# Add dates to the data
for title in tqdm(papers_data.keys(), desc="Fetching paper dates", unit="paper"):
    paper_info = papers_data[title]
    date, arxiv_id = fetch_arxiv_date(title)
    if date:
        paper_info['publication_date'] = date
        paper_info['arxiv_id'] = arxiv_id
        # tqdm.write(f"✓ Found date for: {title} ({date})")
    else:
        continue
        # tqdm.write(f"✗ Could not find date for: {title}")
    # Respect rate limits
    time.sleep(2)

# Save the updated data
with open('papers_data_with_dates.json', 'w') as f:
    json.dump(papers_data, f, indent=4)

print("\nDone! Checking results...")
# Print summary of papers without dates
missing_dates = [title for title, info in papers_data.items() 
                if 'publication_date' not in info]
if missing_dates:
    print(f"\nCould not find dates for {len(missing_dates)} papers:")
    for title in missing_dates:
        print(f"- {title}")
else:
    print("\nFound dates for all papers!")

Fetching paper dates: 100%|████████████| 4886/4886 [6:25:52<00:00,  4.74s/paper]


Done! Checking results...

Could not find dates for 330 papers:
- Energy-Based Diffusion Language Models for Text Generation
- Advancing GenAI Assisted Programming-A Comparative Study on Prompt Efficiency and Code Quality Between GPT-4 and GLM-4
- Meaningful Learning Advancing Abstract Reasoning in Large Language Models via Generic Fact Guidance
- Autonomous Tree-search Ability of Large Language Models
- Retrieval-Augmented Generation for AI-Generated Content A Survey
- Self-Consistency Preference Optimization
- Language Agents Meet Causality - Bridging LLMs and Causal World Models
- Ambiguity-Aware In-Context Learning with Large Language Models
- AVA Towards Autonomous Visualization Agents through Visual PerceptionDriven DecisionMaking
- Prompt Perturbation in Retrieval-Augmented Generation based Large Language Models
- Self-Supervised Multimodal Learning A Survey
- Non-myopic Generation of Language Models for Reasoning and Planning
- Towards More Effective Table-to-Text Generation A

In [15]:
import json
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
from collections import Counter
import re
import numpy as np
from matplotlib.dates import DateFormatter
import matplotlib.dates as mdates
from scipy import stats

def normalize_model_name(name: str) -> tuple:
    """
    Normalizes model names while preserving important distinctions between model sizes
    and architectural variants. Returns tuple of (normalized_name, original_name)
    """
    original = name.strip('"').strip()
    normalized = original.lower()
    normalized = re.sub(r'\s+', ' ', normalized)
    
    # Extract parameter count/size information
    size_pattern = r'[\(\[\s]?(\d+[bm])[\)\]\s]?'
    size_match = re.search(size_pattern, normalized, re.IGNORECASE)
    model_size = size_match.group(1).lower() if size_match else None
    
    # First check for GPT-4 variants
    gpt4_patterns = [
        'gpt4', 'gpt 4', 'gpt-4', 'gpt-4-turbo', 'gpt4-turbo',
        'chatgpt-4', 'chat-gpt-4', 'chatgpt-4.0', 'chat gpt 4', 
        'chatgpt 4', 'gpt-4 turbo', 'gpt 4 turbo'
    ]
    
    if any(re.match(f'^{re.escape(pattern)}', normalized) for pattern in gpt4_patterns):
        if any(v in normalized for v in ['vision', '-v', 'v']):
            return 'gpt-4v', original
        elif any(o in normalized for o in ['4o', '4-o', '4 o']):
            return 'gpt-4o', original
        return 'gpt-4', 'GPT-4'
    
    # Model family mappings with their variants
    base_models = {
        'gpt-3.5': [
            'gpt-3.5-turbo', 'gpt3.5-turbo', 'gpt3.5 turbo', 
            'gpt-3.5-t', 'chatgpt', 'chat-gpt', 'chat gpt', 
            'chatgpt-3.5', 'chatgpt-3.5-turbo', 'chat-gpt-3.5',
            'chat gpt 3.5', 'chatgpt 3.5', 'gpt3.5'
        ],
        'gpt-3': [
            'gpt-3', 'gpt3', 'gpt 3', 'gpt-3-davinci',
            'davinci', 'text-davinci', 'davinci-002',
            'chatgpt-3', 'chat-gpt-3', 'gpt-3-(175b)',
            'gpt-3 (175b)', 'gpt-3 (davinci)'
        ],
        'llama-2': [
            'llama2', 'llama 2', 'llama-2', 'llama2-', 
            'llama-2-', 'llama 2 ', 'llama2 ', 'llama-2 '
        ],
        'llama': [
            'llama1', 'llama 1', 'llama-1', 'llama-', 
            'llama ', 'llamab', 'llama-b'
        ],
        'mistral': [
            'mistral', 'mistral-', 'mistral '
        ],
        'palm': ['palm-', 'palm2', 'palm 2', 'palm-2'],
        'bert': ['bert-base', 'bert base', 'bert-large', 'bert large'],
        'roberta': ['roberta-base', 'roberta base', 'roberta-large', 'roberta large'],
        't5': ['t5-', 't5 ', 't5base', 't5-base', 't5 base']
    }
    
    # Check for other model matches
    for family, variants in base_models.items():
        patterns = [f'^{re.escape(v)}' for v in variants + [family]]
        if any(re.match(pattern, normalized) for pattern in patterns):
            if family in ['llama', 'llama-2', 'mistral']:
                base = family
                
                if any(v in normalized for v in ['instruct', 'ins']):
                    base = f"{base}-instruct"
                elif 'chat' in normalized:
                    base = f"{base}-chat"
                    
                if family == 'mistral' and model_size:
                    return f"{base}-{model_size}", original
                elif model_size:
                    return f"{base}-{model_size}", original
                    
                return base, original
            
            if model_size and family in ['bert', 'roberta', 't5']:
                return f"{family}-{model_size}", original
            
            return family, original
    
    # Remove parenthetical details and extra whitespace
    normalized = re.sub(r'\s*\([^)]*\)\s*', '', normalized)
    normalized = re.sub(r'\s*\[[^\]]*\]\s*', '', normalized)
    normalized = re.sub(r'[-_\s]+', '-', normalized)
    
    if model_size and not normalized.endswith(model_size):
        normalized = f"{normalized}-{model_size}"
        
    return normalized, original

def plot_ridgeline(ax, df, names, counter, colors, vertical_spacing, original_names=None):
    """Plot the ridgeline distribution"""
    # Set fixed date range
    date_min = datetime(2022, 1, 1)
    date_max = datetime(2024, 12, 31)
    
    months = pd.date_range(start=date_min, end=date_max, freq='M')
    month_width = (months[1] - months[0])
    
    monthly_totals = df.groupby(df['date'].dt.to_period('M'))['name'].value_counts()
    max_freq = 0
    
    for idx, name in enumerate(reversed(names)):
        if name in monthly_totals.unstack():
            monthly_counts = monthly_totals.unstack()[name].fillna(0)
            monthly_sums = monthly_totals.groupby('date').sum()
            monthly_freq = monthly_counts / monthly_sums
            max_freq = max(max_freq, monthly_freq.max())
            
            plot_dates = pd.to_datetime([period.to_timestamp() for period in monthly_freq.index])
            baseline = idx * vertical_spacing
            
            # Plot filled bars
            ax.bar(plot_dates, 
                  monthly_freq,
                  width=month_width,
                  bottom=baseline,
                  color=colors[min(idx, len(colors)-1)],
                  alpha=0.6,
                  align='edge',
                  zorder=idx)
            
            # Add border
            ax.bar(plot_dates,
                  monthly_freq,
                  width=month_width,
                  bottom=baseline,
                  color='none',
                  edgecolor=colors[min(idx, len(colors)-1)],
                  linewidth=1.0,
                  alpha=0.9,
                  align='edge',
                  zorder=idx)
            
            # Add peak value label
            max_month_idx = monthly_freq.argmax()
            max_month_date = monthly_freq.index[max_month_idx].to_timestamp()
            ax.text(max_month_date, 
                   baseline + monthly_freq[max_month_idx], 
                   f'{monthly_freq[max_month_idx]:.2f}',
                   ha='left', va='bottom', fontsize=8)
            
            # Add horizontal line
            ax.axhline(y=baseline,
                      color='gray',
                      linewidth=0.2,
                      alpha=0.3,
                      zorder=0)
            
            # Add labels on the right side
            display_name = original_names.get(name, name) if original_names else name
            ax.annotate(f"{display_name} ({counter[name]})",
                       xy=(date_max, baseline),
                       xytext=(5, 0),
                       textcoords="offset points",
                       va='center',
                       fontsize=9,
                       fontfamily='serif',
                       fontstyle='normal',
                       bbox=dict(facecolor='white',
                               edgecolor='none',
                               alpha=0.9,
                               pad=1))

def create_combined_plot(df, names, counter, title, filename, color_h, original_names=None):
    """Create a combined plot with paper counts and distribution"""
    plt.style.use('default')
    plt.rcParams.update({
        'font.family': 'serif',
        'font.size': 10,
        'axes.labelsize': 11,
        'axes.titlesize': 12,
        'figure.titlesize': 14
    })

    fig = plt.figure(figsize=(12, 10))
    
    # Create top subplot for paper counts (25% of height)
    ax1 = plt.subplot2grid((4, 1), (0, 0), rowspan=1)
    
    # Get total mentions per month
    monthly_mentions = df.groupby(df['date'].dt.to_period('M')).size()
    
    start_date = datetime(2022, 1, 1)
    end_date = datetime(2024, 12, 31)
    
    # First, filter papers_df to the date range we want
    papers_df_filtered = papers_df[
        (papers_df.index >= start_date) & 
        (papers_df.index <= end_date)
    ]
    
    # Create date range and align data
    monthly_dates = pd.date_range(start=start_date, end=end_date, freq='M')
    
    # Only include months where we actually have papers
    valid_months = papers_df_filtered.index.to_period('M')
    monthly_mentions = monthly_mentions[monthly_mentions.index.isin(valid_months)]
    
    # Resample papers data
    monthly_papers = papers_df_filtered.resample('M')['total_papers'].last()
    
    # Ensure both series align with actual paper months
    monthly_papers = monthly_papers[monthly_papers > 0]
    monthly_mentions = monthly_mentions[monthly_mentions.index.isin(monthly_papers.index.to_period('M'))]
    
    # Create date range for plotting
    start_date = datetime(2022, 1, 1)
    end_date = datetime(2024, 12, 31)
    monthly_dates = pd.date_range(start=start_date, end=end_date, freq='MS')  # MS = Month Start
    
    # Filter and align the data
    papers_df_filtered = papers_df[
        (papers_df.index >= start_date) & 
        (papers_df.index <= end_date)
    ]
    
    # Resample to month start frequency
    monthly_papers = papers_df_filtered.resample('MS')['total_papers'].last()
    monthly_mentions = df.groupby(df['date'].dt.to_period('M')).size()
    
    # Ensure both series align with actual paper months
    monthly_papers = monthly_papers[monthly_papers > 0]
    monthly_mentions = monthly_mentions[monthly_mentions.index.isin(monthly_papers.index.to_period('M'))]
    
    # Convert period index to timestamp for plotting
    plot_dates = pd.date_range(start=monthly_papers.index.min(), 
                             end=monthly_papers.index.max(), 
                             freq='MS')
    
    # Plot paper counts - align to start of each month
    bars = ax1.bar(plot_dates,
                   monthly_papers.values,
                   width=30,
                   color='gray',
                   alpha=0.5,
                   align='edge')  # Align to start of month

    # Customize top subplot
    ax1.set_xlim(start_date, end_date)
    ax1.spines['top'].set_visible(False)
    
    # Add value labels for paper counts
    for bar in bars:
        height = bar.get_height()
        if height > 0:
            ax1.text(bar.get_x() + bar.get_width()/2, height,
                    f'{int(height)}',
                    ha='center', va='bottom', fontsize=8)
    
    # Create bottom subplot for distribution (75% of height)
    ax2 = plt.subplot2grid((4, 1), (1, 0), rowspan=3)
    
    n_shades = len(names)
    vertical_spacing = 0.35
    colors = sns.husl_palette(n_shades, h=color_h, s=0.85, l=0.6)
    
    plot_ridgeline(ax2, df, names, counter, colors, vertical_spacing, original_names)
    
    ax2.set_xlim(start_date, end_date)
    ax2.yaxis.set_visible(False)
    ax2.spines['top'].set_visible(False)
    ax2.spines['right'].set_visible(False)
    ax2.spines['left'].set_visible(False)
    
    # Format x-axis with year ticks
    for ax in [ax1, ax2]:
        # Set major ticks only at start of year
        ax.xaxis.set_major_locator(mdates.YearLocator(month=1, day=1))
        
        # Remove year labels from ticks
        ax.xaxis.set_major_formatter(plt.NullFormatter())
        
        # Handle year labels differently for each subplot
        if ax == ax1:  # Top subplot
            y_pos = ax.get_ylim()[0] - 0.1
        else:  # Bottom subplot (ax2)
            y_pos = -0.02  # Reduced negative value to move labels up
            
        # Manually add year labels at the middle of each year
        years = range(2022, 2025)
        for year in years:
            mid_year_date = datetime(year, 7, 1)
            ax.text(mdates.date2num(mid_year_date), y_pos,
                   str(year),
                   ha='center',
                   va='top',
                   transform=ax.get_xaxis_transform())
        
        # Add minor ticks for all months
        ax.xaxis.set_minor_locator(mdates.MonthLocator())
        
        # Customize tick lengths
        ax.tick_params(axis='x', which='major', length=8)  # Longer ticks for years
        ax.tick_params(axis='x', which='minor', length=4)  # Shorter ticks for months
        
        # Adjust label positions to center them
        ax.tick_params(axis='x', which='major', pad=10)  # Add padding for year labels
        
        # Add subtle grid
        ax.grid(True, alpha=0.1, linestyle='-', axis='x')
        ax.grid(True, alpha=0.05, linestyle='-', axis='x', which='minor')
    
    # Only show x-label on bottom plot
    ax1.set_xlabel('')
    ax2.set_xlabel('')
    
    # Adjust layout
    plt.subplots_adjust(right=0.85, hspace=0.3)
    
    plt.savefig(filename,
                dpi=300,
                bbox_inches='tight',
                facecolor='white')
    plt.close()

# Load and process data
with open('papers_data_with_dates.json', 'r') as f:
    papers_data = json.load(f)

# Calculate total papers per month
all_papers_by_month = {}
for title, paper_info in papers_data.items():
    if 'publication_date' in paper_info:
        date = paper_info['publication_date']
        date_obj = datetime.strptime(date, '%Y-%m-%d')
        month_key = date_obj.strftime('%Y-%m')
        all_papers_by_month[month_key] = all_papers_by_month.get(month_key, 0) + 1

# Convert to DataFrame for plotting
papers_df = pd.DataFrame(
    [(datetime.strptime(k, '%Y-%m'), v) for k, v in all_papers_by_month.items()],
    columns=['date', 'total_papers']
)
papers_df.set_index('date', inplace=True)

# Get frequency counts
model_counter = Counter()
benchmark_counter = Counter()
model_original_names = {}
benchmark_original_names = {}
benchmark_records = []
model_records = []

# Process the data
for title, paper_info in papers_data.items():
    if 'publication_date' in paper_info and 'analysis' in paper_info:
        date = paper_info['publication_date']
        date_obj = datetime.strptime(date, '%Y-%m-%d')
        
        # Only process data from 2022 onwards
        if date_obj.year >= 2022:
            # Process benchmarks
            if 'benchmarks' in paper_info['analysis']:
                for benchmark in paper_info['analysis']['benchmarks']:
                    normalized_benchmark = benchmark.strip('"').strip().lower()
                    normalized_benchmark = re.sub(r'[-_\s]+', '-', normalized_benchmark)
                    benchmark_counter[normalized_benchmark] += 1
                    benchmark_original_names[normalized_benchmark] = benchmark.strip('"').strip()
                    benchmark_records.append({
                        'date': date_obj,
                        'name': normalized_benchmark,
                        'type': 'Benchmark',
                        'paper': title
                    })
            
            # Process models
            if 'base_models' in paper_info['analysis']:
                for model in paper_info['analysis']['base_models']:
                    normalized_model, original_name = normalize_model_name(model)
                    model_counter[normalized_model] += 1
                    if normalized_model not in model_original_names:
                        model_original_names[normalized_model] = original_name
                    model_records.append({
                        'date': date_obj,
                        'name': normalized_model,
                        'type': 'Model',
                        'paper': title
                    })

# Get top 20 benchmarks and models
top_20_benchmarks = [b[0] for b in benchmark_counter.most_common(20)]
top_20_models = [m[0] for m in model_counter.most_common(20)]

# Create DataFrames
df_benchmarks = pd.DataFrame(benchmark_records)
df_models = pd.DataFrame(model_records)

# Filter for top 20
df_benchmarks = df_benchmarks[df_benchmarks['name'].isin(top_20_benchmarks)]
df_models = df_models[df_models['name'].isin(top_20_models)]

# Update DataFrames to use datetime
df_benchmarks['date'] = pd.to_datetime(df_benchmarks['date'])
df_models['date'] = pd.to_datetime(df_models['date'])

# Calculate statistics
total_papers = len(set(title for title, paper_info in papers_data.items() 
                      if 'publication_date' in paper_info 
                      and datetime.strptime(paper_info['publication_date'], '%Y-%m-%d').year >= 2022))

total_model_mentions = 0
total_benchmark_mentions = 0
papers_with_models = 0
papers_with_benchmarks = 0

for title, paper_info in papers_data.items():
    if 'publication_date' in paper_info and 'analysis' in paper_info:
        date = datetime.strptime(paper_info['publication_date'], '%Y-%m-%d')
        if date.year >= 2022:
            # Count model mentions
            if 'base_models' in paper_info['analysis'] and paper_info['analysis']['base_models']:
                papers_with_models += 1
                total_model_mentions += len(paper_info['analysis']['base_models'])
            
            # Count benchmark mentions
            if 'benchmarks' in paper_info['analysis'] and paper_info['analysis']['benchmarks']:
                papers_with_benchmarks += 1
                total_benchmark_mentions += len(paper_info['analysis']['benchmarks'])

# Calculate averages
avg_models_per_paper = total_model_mentions / total_papers
avg_benchmarks_per_paper = total_benchmark_mentions / total_papers
avg_models_when_present = total_model_mentions / papers_with_models if papers_with_models > 0 else 0
avg_benchmarks_when_present = total_benchmark_mentions / papers_with_benchmarks if papers_with_benchmarks > 0 else 0

print("\nPaper Statistics:")
print(f"Total papers analyzed: {total_papers}")
print(f"\nModel statistics:")
print(f"Total model mentions: {total_model_mentions}")
print(f"Papers that mention models: {papers_with_models} ({papers_with_models/total_papers*100:.1f}%)")
print(f"Average models per paper (across all papers): {avg_models_per_paper:.2f}")
print(f"Average models when present: {avg_models_when_present:.2f}")

print(f"\nBenchmark statistics:")
print(f"Total benchmark mentions: {total_benchmark_mentions}")
print(f"Papers that mention benchmarks: {papers_with_benchmarks} ({papers_with_benchmarks/total_papers*100:.1f}%)")
print(f"Average benchmarks per paper (across all papers): {avg_benchmarks_per_paper:.2f}")
print(f"Average benchmarks when present: {avg_benchmarks_when_present:.2f}")

# Create combined plots
create_combined_plot(df_benchmarks, top_20_benchmarks, benchmark_counter,
                    'Distribution of Benchmark Usage Over Time',
                    'benchmark_distribution.pdf', 0.5,
                    benchmark_original_names)

create_combined_plot(df_models, top_20_models, model_counter,
                    'Distribution of Model Usage Over Time',
                    'model_distribution.pdf', 0.9,
                    model_original_names)

# Print statistics
print("\nTop 20 Benchmarks and their frequencies:")
for benchmark, count in benchmark_counter.most_common(20):
    original_name = benchmark_original_names.get(benchmark, benchmark)
    print(f"{original_name}: {count}")

print("\nTop 20 Models and their frequencies:")
for model, count in model_counter.most_common(20):
    original_name = model_original_names.get(model, model)
    print(f"{original_name}: {count}")


Paper Statistics:
Total papers analyzed: 4865

Model statistics:
Total model mentions: 16629
Papers that mention models: 4760 (97.8%)
Average models per paper (across all papers): 3.42
Average models when present: 3.49

Benchmark statistics:
Total benchmark mentions: 14903
Papers that mention benchmarks: 4464 (91.8%)
Average benchmarks per paper (across all papers): 3.06
Average benchmarks when present: 3.34

Top 20 Benchmarks and their frequencies:
GSM8K: 425
MATH: 174
MMLU: 171
SVAMP: 139
StrategyQA: 126
HotpotQA: 120
HumanEval: 119
TruthfulQA: 81
CommonsenseQA: 77
HellaSwag: 70
AQUA: 64
TriviaQA: 63
MBPP: 63
MultiArith: 61
Winogrande: 56
BoolQ: 55
PiQA: 52
openbookqa: 49
ASDiv: 46
SQuAD: 44

Top 20 Models and their frequencies:
GPT-4: 1789
ChatGPT (gpt-3.5-turbo): 1739
GPT-3 (DaVinci-002): 705
PaLM-2: 415
GPT4o: 373
LLaMa 2 (7B parameters): 372
BERT: 266
Llama-3-8B: 264
Mistral 7B: 243
LLaMA-1: 230
LLaMA-7B: 201
LLAMA-2 (13b): 198
GPT-4V: 195
Llama3 (70B): 175
LLaMA 2 70B: 172
T5: 